In [1]:
# from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import numpy as np
import math

In [2]:
import data_file
# male_list = data_file.male_list
# female_list = data_file.female_list
female_professions = data_file.female_professions
male_professions = data_file.male_professions

In [5]:
# models

# model1
from transformers import BertTokenizer, BertModel
import torch
tokenizer1 = BertTokenizer.from_pretrained('bert-base-uncased')
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model1 = BertModel.from_pretrained("bert-base-uncased")
# model1.cuda()

# model2
from transformers import BertTokenizer, BertModel
import torch
tokenizer2 = BertTokenizer.from_pretrained('bert-large-uncased')
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model2 = BertModel.from_pretrained("bert-large-uncased")
# model2.cuda()

# model3
from transformers import RobertaTokenizer, RobertaModel
import torch
tokenizer3 = RobertaTokenizer.from_pretrained('roberta-base')
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model3 = RobertaModel.from_pretrained("roberta-base")
# model3.cuda()

# model4
from transformers import DistilBertTokenizer, DistilBertModel
import torch
tokenizer4 = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model4 = DistilBertModel.from_pretrained("distilbert-base-uncased")
# model4.cuda()

# model5
from transformers import BertTokenizer, BertModel
import torch
tokenizer5 = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model5 = BertModel.from_pretrained("bert-base-multilingual-uncased")
# model5.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.t

In [6]:
models = [model1, model2, model3, model4, model5]
tokenizers = [tokenizer1, tokenizer2, tokenizer3, tokenizer4, tokenizer5]

In [7]:
female_list = list(female_professions.label1)
male_list = list(male_professions.label1)

In [8]:
# # gendered_word_list
# gendered_word_list = [["she", "he"],["female", "male"], ["woman", "man"], ["girl", "boy"], ["sister", "brother"], ["daughter", "son"], ["mother", "father"]]
XX = ["female", "woman", "girl", "sister", "daughter", "mother", "aunt", "grandmother", "daughter"]
YY = ["male", "man", "boy", "brother", "son", "father", "uncle", "grandfather", "son"]

In [9]:
templates = ["this is the _.", "that is the _.", "there is the _.", "here is the _.", "the _ is here.", "the _ is there.", "the _ is a person.", "it is the _.","the _ is a [MASK].", "the _ is an engineer.", "the _ is a nurse with superior technical skills."]

In [10]:
temps = ["temp1", "temp2", "temp3", "temp4","temp5", "temp6", "temp7", "temp8", "temp9", "temp10", "temp11"]

In [11]:
len(templates)

11

In [12]:
# CLS embedding
def sentence_embedding(template, word, model_ind):
    sentence = template.replace("_", word)
    inputs = tokenizers[model_ind](sentence, return_tensors="pt")
    outputs = models[model_ind](**inputs)
    last_hidden_states = outputs.last_hidden_state
    token_embeddings = last_hidden_states
    return token_embeddings[0][0].cpu().detach().numpy()

In [13]:
def cossim(x, y):
    return np.dot(x, y) / math.sqrt(np.dot(x, x) * np.dot(y, y))

In [14]:
def construct_cossim_lookup(XY, AB):
    """
    XY: mapping from target string to target vector (either in X or Y)
    AB: mapping from attribute string to attribute vectore (either in A or B)
    Returns an array of size (len(XY), len(AB)) containing cosine similarities
    between items in XY and items in AB.
    """

    cossims = np.zeros((len(XY), len(AB)))
    for xy in XY:
        for ab in AB:
            cossims[xy, ab] = cossim(XY[xy], AB[ab])
    return cossims


In [15]:
def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )

In [16]:

def s_XAB(A, s_wAB_memo):
    return s_wAB_memo[A].sum()

In [17]:
def s_wAB(X, Y, cossims):
    """
    Return vector of s(w, A, B) across w, where
        s(w, A, B) = mean_{a in A} cos(w, a) - mean_{b in B} cos(w, b).
    """
#     print((cossims[X, :].mean(axis=0) - cossims[Y, :].mean(axis=0)).shape)
    return cossims[X, :].mean(axis=0) - cossims[Y, :].mean(axis=0)

In [18]:
# def s_wAB_df(X, Y, cossims):
#     """
#     Return vector of s(w, A, B) across w, where
#         s(w, A, B) = mean_{a in A} cos(w, a) - mean_{b in B} cos(w, b).
#     """
# #     print(cossims[:, A].mean(axis=1))
#     df1 = pd.DataFrame(cossims[X, :].mean(axis=0))
#     df2 = pd.DataFrame(cossims[Y, :].mean(axis=0))
#     return df1, df2


def s_XAB_df(A, B, s_wAB_memo):
    df1 = pd.DataFrame(s_wAB_memo[A])
    df2 = pd.DataFrame(s_wAB_memo[B])
    return df1, df2

In [19]:
def s_XYAB(A, B, s_wAB_memo):
    r"""
    Given indices of target concept X and precomputed s_wAB values,
    the WEAT test statistic for p-value computation.
    """
    return s_XAB(A, s_wAB_memo) - s_XAB(B, s_wAB_memo)

In [20]:
# def WEAT_test(X, Y, A, B, n_samples, cossims, parametric=False):
def WEAT_test(X, Y, A, B, n_samples, cossims):
    ''' Compute the p-val for the permutation test, which is defined as
        the probability that a random even partition X_i, Y_i of X u Y
        satisfies P[s(X_i, Y_i, A, B) > s(X, Y, A, B)]
    '''
    X = np.array(list(X), dtype=np.int)
    Y = np.array(list(Y), dtype=np.int)
    A = np.array(list(A), dtype=np.int)
    B = np.array(list(B), dtype=np.int)

    assert len(X) == len(Y)
    size = len(X)
    s_wAB_memo = s_wAB(X, Y, cossims=cossims)
#     print(s_wAB_memo)
    XY = np.concatenate((X, Y))

#     if parametric:
#     log.info('Using parametric test')
    s = s_XYAB(A, B, s_wAB_memo)
    return s


def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )

In [21]:
def get_effect_size(df1, df2, k=0):
    diff = (df1[k].mean() - df2[k].mean())
    std_ = pd.concat([df1, df2], axis=0)[k].std() + 1e-8
    return diff / std_

In [23]:
template_score_dict = {}
for ind, template in enumerate(templates):
    score_dict = {}
    attribute_template = template
    target_template = template
    for model_ind in range(len(models)):
        X = {"x" + str(j): sentence_embedding(attribute_template, j, model_ind) for j in XX}
        Y = {"y" + str(j): sentence_embedding(attribute_template, j, model_ind) for j in YY}
        (X, Y) = convert_keys_to_ints(X, Y)
        XY = X.copy()
        XY.update(Y)
        X = np.array(list(X), dtype=np.int)
        Y = np.array(list(Y), dtype=np.int)
        AA = female_list
    #     print(AA)
    #     print(XX)
        BB = male_list

        A = {"a" + str(j): sentence_embedding(target_template, j, model_ind) for j in AA}
        B = {"b" + str(j): sentence_embedding(target_template, j, model_ind) for j in BB}

        (A, B) = convert_keys_to_ints(A, B)


        AB = A.copy()
        AB.update(B)

        cossims = construct_cossim_lookup(XY, AB)
        A = np.array(list(A), dtype=np.int)
        B = np.array(list(B), dtype=np.int)


        s_wAB_memo = s_wAB(X, Y, cossims=cossims)
        df1,df2 = s_XAB_df(A, B, s_wAB_memo)
        effect_size = get_effect_size(df1, df2)
        score_dict[model_ind] = effect_size
        print(score_dict[model_ind])
    template_score_dict[temps[ind]] = score_dict
# log.info("pval: %g", pval)

# log.info("computing effect size...")
# esize = effect_size(X, Y, A, B, cossims=cossims)
# log.info("esize: %g", esize)

1.167137765206161
0.613181978410407
1.2450650026597185
0.8241778491485858
0.3899740677999021
1.1429489272509046
0.7682580209621059
1.0448924399341275
0.871382442573088
0.15952797256527132
0.6468110250285095
0.23944622026258083
1.1923981150296132
0.41716703855199594
0.37244794398602465
1.1573514938240603
0.5820993930584908
1.0615581872798616
0.7615882028614193
0.7578026564668334
0.9910124198202783
0.40116426444909276
1.1237046148444554
0.7650705113511841
0.3550183256840251
1.0536814040106905
0.5898308774572396
1.1864499464559901
0.8110554706993732
0.3941374836136234
0.6099462158905813
0.3150538995972494
1.2670202061475453
0.8481557113873969
0.3820282424854507
1.19501511543039
0.7900532245667686
1.159718639716848
0.7631024037307255
0.41156324117274534
1.0619592517147627
0.4723792698741765
1.1050926948724156
1.0947279057120534
0.16922043676379692
0.3759700281077367
0.666829483801519
1.3187638450729833
0.8741564533514405
0.5007626048939062
0.5334231945190242
0.2554062954334604
0.9558810376

In [24]:
import json
a_file = open("seat_CLS_templates_Language_models.json", "w")
json.dump(template_score_dict, a_file)
a_file.close()